In [ ]:
!pip install -q -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.7 MB/s eta 0:00:00


In [ ]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
from google.colab import userdata

# Access the API key from Colab's secrets manager
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=GEMINI_API_KEY)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make predictions or decisions.


In [ ]:
! pip install langchain
! pip install langchain_google_genai
! pip install langchain_community
! pip install chromadb
! pip install pypdf
! pip install tiktoken

# Task
Create a RAG + Gemini code to embed the document "/content/2023년+동물용의약품등+품목+등록현황 (1).csv".

## Load and preprocess the data

### Subtask:
Load the CSV file into a pandas DataFrame and preprocess it for RAG.


**Reasoning**:
Import pandas and load the CSV file into a DataFrame, handling potential encoding issues. Display the head and info of the DataFrame to inspect it.



In [2]:
import pandas as pd

try:
    df = pd.read_csv('/content/2023년+동물용의약품등+품목+등록현황 (1).csv', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv('/content/2023년+동물용의약품등+품목+등록현황 (1).csv', encoding='cp949')

display(df.head())
display(df.info())

,용도,허가번호,업종,품목명,성분명(국문),성분명(영문),허가일,업체명
0,동물용의약품,001-003,수입,교토비켄 케나인-9(KYOTO BIKEN CANINE-9),디스템퍼+전염성간염+파라인플+파보바+렙토스피라+코로나,DHPPL+Coronavirus,2003-08-07,(주)삼우메디안
1,동물용의약품,002-003,제조,슈퍼마이신,바시트라신 아연,Zn-Bacitracin,1963-05-13,(주)우성양행
2,동물용의약품,002-004,제조,골든종계용(GOLDEN B.D),기타영양공급약,Mineral supplemental preparations,1980-02-19,(주)우성양행
3,동물용의약품,002-005,제조,골든육계용(GOLDEN B.R),기타영양공급약,Mineral supplemental preparations,1980-02-19,(주)우성양행
4,동물용의약품,002-006,제조,골든팜,기타영양공급약,Mineral supplemental preparations,1980-02-19,(주)우성양행


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11851 entries, 0 to 11850
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   용도       11851 non-null  object
 1   허가번호     11851 non-null  object
 2   업종       11851 non-null  object
 3   품목명      11851 non-null  object
 4   성분명(국문)  11851 non-null  object
 5   성분명(영문)  11829 non-null  object
 6   허가일      11790 non-null  object
 7   업체명      11851 non-null  object
dtypes: object(8)
memory usage: 740.8+ KB


None

## Create document embeddings

### Subtask:
Generate embeddings for the document content using a suitable embedding model.


**Reasoning**:
Generate embeddings for the document content using a suitable embedding model.



In [ ]:
!pip install -U langchain-huggingface
!pip install -U sentence-transformers transformers tokenizers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.7.3 requires tokenizers<0.21.0,>=0.20.0, but you have tokenizers 0.22.0 which is incompatible.


In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from google.colab import userdata
import os

# Colab userdata에서 토큰 가져오기, 없으면 오류 표시
HF_TOKEN = userdata.get('HF_TOKEN')
if not HF_TOKEN:
    raise ValueError("Hugging Face API 토큰이 userdata에 없습니다. 환경 변수에 꼭 등록하세요.")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

# 텍스트 분할기 설정
texts = [
    "안녕하세요, 만나서 반갑습니다.",
    "LangChain은 대규모 언어 모델을 활용한 애플리케이션 개발을 쉽게 합니다.",
    "한국어 자연어 처리 분야에서 임베딩은 매우 중요한 기술입니다.",
    "Retrieval-Augmented Generation (RAG)은 AI 응답의 품질을 높이는 기법입니다.",
    "이 문장은 테스트용 예시 문장입니다.",
    "Artificial intelligence and machine learning are transforming industries.",
    "This is an example sentence for embedding testing.",
    "텍스트 임베딩은 문서 검색과 추천 시스템에 필수적입니다.",
    "딥러닝 기반 자연어 처리 모델의 발전이 계속되고 있습니다.",
    "LangChain simplifies application development using large language models."
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
df_sampled = df.sample(frac=0.001, random_state=42)  # 25% 랜덤 샘플, random_state는 재현성 위해 고정

# 데이터프레임 행 단위 텍스트 병합, 빈 값 무시
df_sampled['text'] = df_sampled.apply(lambda row: ' '.join(str(x) for x in row if pd.notna(x) and str(x).strip()), axis=1)
texts = df_sampled['text'].tolist()

print(df_sampled.head())

# 문서 객체 생성
docs = text_splitter.create_documents(texts)

# 임베딩 모델 설정 및 임베딩 생성
model_name = "intfloat/multilingual-e5-large-instruct"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

try:
    doc_embeddings = embeddings.embed_documents([doc.page_content for doc in docs])
except Exception as e:
    print("임베딩 생성 중 에러 발생:", e)
    doc_embeddings = None


            용도     허가번호  업종                                      품목명  \
10299  동물용의료기기  134-009  제조                  수동식재사용가능의료용핸드피스[1](DVH)   
900     동물용의약품  008-222  제조                   셀락탈 주사액(CELACTAL INJ.)   
1744    동물용의약품  013-158  제조                  펠리샷 피에이치씨(FeliShot PHC)   
9367   동물용의료기기  074-008  수입  요화학검사시약[2](Thinka Urine TestStrip 10UB)   
6712    동물용의약품  283-035  수입                         옵티실드(Optishield)   

                                    성분명(국문)  \
10299                    수동식재사용가능의료용핸드피스[1]   
900                                    자일라진   
1744   고양이범백혈구감소증바이러스 고양이허피스바이러스 고양이칼리시바이러스   
9367                             요화학검사시약[2]   
6712                              사이클로스포린 A   

                                                 성분명(영문)         허가일  \
10299                   Surgical drill handpiece, manual  2016-11-22   
900                                            Xylazines  1987-05-19   
1744   Feline Panleukopenia virus+Feline Herpesvirus+...  2006-06-14

## Set up a vector store

### Subtask:
Initialize a vector store to store the document embeddings for efficient retrieval.


**Reasoning**:
Initialize a Chroma vector store with the documents and embeddings.



In [13]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

## Implement the rag chain

### Subtask:
Set up a RAG chain using Langchain to combine the retrieval and generation components.


**Reasoning**:
Import the necessary classes and define the prompt template, initialize the language model, and create the RAG chain as instructed.



In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

# Access the API key from Colab's secrets manager
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

prompt_template = ChatPromptTemplate.from_template(
    "Answer the question based only on the following context:\n{context}\nQuestion: {question}"
)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=GEMINI_API_KEY)

rag_chain = (
    {"context": vectorstore.as_retriever(), "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

## Implement the rag chain

### Subtask:
Set up a RAG chain using Langchain to combine the retrieval and generation components.


## Query the rag system

### Subtask:
Write code to query the RAG system with user questions and display the generated responses.


**Reasoning**:
Define a query, call the RAG chain to get the response, and print the response.



In [16]:
query = "옵티실드의 성분은 무엇인가요? 그리고 용법은 어떻게 되나요?"
response = rag_chain.invoke(query)
print(response)

옵티실드의 성분은 **사이클로스포린 A (cyclosporin A)** 입니다.

용법에 대한 정보는 주어진 내용에 없습니다.


# Crawling
## Subtesk:
in medi.qia site. search pharms and get info

In [23]:
import requests
from bs4 import BeautifulSoup
import time

BASE_URL = "https://medi.qia.go.kr"
SEARCH_URL = f"{BASE_URL}/searchMedicine"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

def search_medicine(keyword):
    """
    의약품명으로 검색 후 결과 테이블에서 상세페이지 URL과 기본정보 리스트 반환
    """
    params = {
        "itemName": keyword  # 실제 input의 name 값에 맞춰 조정 필요
    }
    res = requests.get(SEARCH_URL, headers=HEADERS, params=params)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    # 테이블 찾기 (첨부 HTML 기준)
    table = soup.find("table")
    results = []
    if table:
        headers = [th.text.strip() for th in table.find_all("th")]

        for tr in table.find_all("tr")[1:]:
          td_list = tr.find_all("td")
          if not td_list:
              continue

          row_data = {headers[i]: td_list[i].text.strip() for i in range(len(td_list))}

          # 제품명 컬럼에서 상세페이지 링크 추출
          product_td = td_list[1]  # 제품명 컬럼 위치 반드시 확인!
          a_tag = product_td.find("a")
          if a_tag and a_tag.get("href"):
              detail_url = BASE_URL + a_tag.get("href")
          else:
              detail_url = None
          row_data["detail_url"] = detail_url

          results.append(row_data)

    return results

def parse_detail_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    container = soup.find('div', class_='drug_info_mid')
    if not container:
        return {}

    # 전체 info_sec 요소를 분류해서 담기
    raw_materials = []
    additives = None
    efficacy = None
    warnings_and_side_effects = None

    info_sections = container.find_all('div', class_='info_sec')
    for section in info_sections:
        text = section.get_text(separator=' ', strip=True)
        # 키워드 포함 여부로 대략 구분 (필요시 더 정교하게 수정)
        if '원료약품' in text or '원료' in text:
            raw_materials.append(text)
        elif '첨가제' in text:
            additives = text
        elif '효능효과' in text:
            efficacy = text
        elif '주의사항' in text or '부작용' in text:
            warnings_and_side_effects = text

    return {
        "raw_materials": raw_materials,
        "additives": additives,
        "efficacy": efficacy,
        "warnings_and_side_effects": warnings_and_side_effects
    }



if __name__ == "__main__":
    keyword = "옵티실드"  # 검색할 의약품명

    # 검색 결과에서 제품 기본정보, 상세페이지 URL 수집
    results = search_medicine(keyword)
    print(f"총 {len(results)}건 검색 완료")

    for idx, item in enumerate(results):
        print(f"\n[{idx +1}] 제품명: {item.get('제품명')}")
        print(f"업체명: {item.get('업체명')}")
        print(f"성분: {item.get('주성분')}")
        print(f"상세페이지: {item.get('detail_url')}")

        # 상세페이지 정보 크롤링
        detail_data = parse_detail_page(item.get('detail_url'))

        if detail_data:
            print("\n원료약품 및 분량:")
            for d in detail_data.get("raw_materials", []):
                print(d)

            print("\n첨가제:")
            print(detail_data.get("additives", "없음"))

            print("\n효능효과:")
            print(detail_data.get("efficacy", "없음"))

            print("\n주의사항 및 부작용:")
            print(detail_data.get("warnings_and_side_effects", "없음"))

        time.sleep(1)  # 서버부하 방지



총 1건 검색 완료

[1] 제품명: 제품명
옵티실드
업체명: 업체명제이에스코리아
성분: 주성분사이클로스포린
상세페이지: https://medi.qia.go.kr/searchMedicine/getItemDetail?itemSeq=202326323

원료약품 및 분량:
원료약품 및 분량 1그램 중 순번 성분명 분량 단위 규격 비고 1 사이클로스포린 2.0 MG USP - 첨가제 : 파라핀,유동파라핀,에탄올

첨가제:
None

효능효과:
효능효과 효능효과 폴딩 버튼 개의 만성 건성각결막염(Chronic Keratoconjunctivitis Sicca, KCS)의 치료

주의사항 및 부작용:
주의사항 주의사항 폴딩 버튼 가 . 다음의 환축에게는 투여하지 말 것 : 선천적 무누증(Congenital Alacrima), 설폰아마이드의 사용, 디스템퍼바이러스 감염증, 대사성 질환, 안면신경마비로 인한 안검반사의 소실로 인해 발생한 각결막염의 경우 투약 효과가 없을 수 있으니 투여하지 말 것. 나 . 부작용 안검주위의 발적 및 심한 가려움증의 경우 제품에 대한 과민반응일 수 있으며, 이러한 증상이 일주일 이상 지속될 경우 투약을 중지해야 함. 다 . 임신 , 수유축 , 신생축 , 어린 가축 , 쇠약축 등에 대한 투여 강아지 및 임신견에서의 안전성은 검증돼있지 않으므로 사용시 주의할 것. 라 . 적용상의 주의사항 눈에 바이러스나 곰팡이성 질병이 이미 감염되어 있는 경우 먼저 질병을 치료한 후 본 제를 사용할 것. 바 . 저장상의 주의사항 : 어린 아이의 손이 닿지 않는 곳에 보관할 것. 사 . 기타 주의사항 (1) 사용설명서를 충분히 읽어본 후 사용하십시오. (2) 수의사의 처방에 따라 사용하십시오. (3) 동물용의약품이므로 사람에게는 절대 사용하지 마십시오. (4) 애완견에만 사용할 것(식육동물에 사용금지). (5) 기타 문의 사항이 있을 시 수입/판매자에게 연락하여 주시기 바랍니다.
